# Separate All Parts (Demucs)

In [13]:
# Add the Demucs directory to the Python path
demucs_dir = os.path.abspath('../../packages/demucs')
sys.path.append(demucs_dir)

In [12]:
import sys
import os

from demucs.separate import main as demucs_separate

# Path to your audio file
audio_file_path = './data/moon_track.mp3'

# Use the Demucs model for separation
# This will create separated tracks for drums, bass, other instruments, and vocals
demucs_separate(['-n', 'mdx_extra', audio_file_path])

# Use the `htdemucs_6s` model for separation, which includes piano and guitar
demucs_separate(['-n', 'htdemucs_6s', audio_file_path])

Selected model is a bag of 1 models. You will see that many progress bars per track.
Separated tracks will be stored in /Users/jingbinc/GitHub/personal/track_separation/separated/htdemucs_6s
Separating track data/moon_track.mp3


100%|████████████████████████████████████████████████████████████████████████| 292.5/292.5 [02:22<00:00,  2.05seconds/s]


# Audio Manipulation

### Separate possible files

In [20]:
import os
import torch
import torchaudio

def audio_manipulation(original_file, subtract_files, output_file):
    """
    Subtract audio content of 'subtract_files' from 'original_file' and save the result.
    
    Parameters:
    original_file (str): The path to the original audio file.
    subtract_files (list): List of paths to audio files that will be subtracted from the original.
    output_file (str): The path where the output file will be saved.
    """
    # Load the original audio
    original_audio, orig_sr = torchaudio.load(original_file)
    
    # Subtract files
    for file in subtract_files:
        # Load the subtracting file
        sub_audio, sub_sr = torchaudio.load(file)
        
        # Resample if necessary
        if sub_sr != orig_sr:
            resampler = torchaudio.transforms.Resample(sub_sr, orig_sr)
            sub_audio = resampler(sub_audio)

        # Trim/Pad the audio to match lengths
        min_length = min(original_audio.shape[1], sub_audio.shape[1])
        original_audio = original_audio[:, :min_length]
        sub_audio = sub_audio[:, :min_length]

        # Subtract the audio content
        original_audio -= sub_audio

    # Save the resulting audio
    torchaudio.save(output_file, original_audio, orig_sr)

    print(f"Processed audio saved as {output_file}")

In [21]:
# Example usage: Separate vocal, piano, and other
original_file = './data/moon_track.mp3'
subtract_files = ['./separated/htdemucs_6s/moon_track/other.wav', 
                  './separated/htdemucs_6s/moon_track/vocals.wav', 
                  './separated/htdemucs_6s/moon_track/piano.wav']
output_file = './data/moon_track_minus_components_v1.wav'

audio_manipulation(original_file, subtract_files, output_file)

Processed audio saved as ./data/moon_track_minus_components_v1.wav


### Separate part files

In [17]:
import torch
import torchaudio

# Define file paths
original_file = './data/moon_track.mp3'
piano_file = './separated/htdemucs_6s/moon_track/piano.wav'
output_file = './data/moon_track_minus_piano_v2.wav'

# Load the original and piano tracks
original_audio, orig_sr = torchaudio.load(original_file)
piano_audio, piano_sr = torchaudio.load(piano_file)

# Resample if sample rates are different
if orig_sr != piano_sr:
    resampler = torchaudio.transforms.Resample(orig_sr, piano_sr)
    original_audio = resampler(original_audio)

# Making sure both tracks are of the same length
min_length = min(original_audio.shape[1], piano_audio.shape[1])
original_audio = original_audio[:, :min_length]
piano_audio = piano_audio[:, :min_length]

# Subtract the piano from the original track
result_audio = original_audio - piano_audio

# Save the resulting audio
torchaudio.save(output_file, result_audio, orig_sr)

print(f"Processed audio saved as {output_file}")


Processed audio saved as ./data/moon_track_minus_piano_v2.wav
